<a href="https://www.kaggle.com/code/tirendazacademy/emotion-analysis-with-bert?scriptVersionId=146438765" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# <b><div style='padding:15px;background-color:#850E35;color:white;border-radius:2px;font-size:110%;text-align: center'> Emotion Analysis with TensorFlow, Hugging Face and BERT </div></b>

![](https://img.freepik.com/free-vector/girl-face-avatar-with-different-emotion-expression_107791-19830.jpg?w=1380&t=st=1697191626~exp=1697192226~hmac=cd3a951df743ba2f0fc074031cad0402884b67e8bc77f68947b375c0021208d5)

Hi guys 😀 In this notebook, we're going to cover how to perform sentiment analysis with Hugging Face and TensorFlow. Let's get started.

# <b><div style='padding:15px;background-color:#850E35;color:white;border-radius:2px;font-size:110%;text-align: center'>1. Dataset Loading </div></b>

In [ ]:
!pip install -q datasets

In [ ]:
from datasets import load_dataset
emotions = load_dataset("dair-ai/emotion")
emotions

# <b><div style='padding:15px;background-color:#850E35;color:white;border-radius:2px;font-size:110%;text-align: center'>2. Understanding the Dataset </div></b>

In [ ]:
emotions["train"][0]

In [ ]:
emotions["train"].column_names

In [ ]:
emotions["train"].features

# <b><div style='padding:15px;background-color:#850E35;color:white;border-radius:2px;font-size:110%;text-align: center'>3. Data Preprocessing </div></b>

In [ ]:
from transformers import AutoTokenizer
model_ckpt = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [ ]:
def tokenize(examples):
    return tokenizer(examples["text"], truncation=True)

In [ ]:
emotions_encoded = emotions.map(tokenize, batched=True, batch_size=None)

## Padding

In [ ]:
from transformers import DataCollatorWithPadding

In [ ]:
data_collator=DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

## Evaluate

In [ ]:
!pip install -q evaluate

In [ ]:
import evaluate
accuracy = evaluate.load("accuracy")

In [ ]:
import numpy as np 

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, 
                            references = labels)

# <b><div style='padding:15px;background-color:#850E35;color:white;border-radius:2px;font-size:110%;text-align: center'>4. Model Training </div></b>

In [ ]:
from transformers import TFAutoModelForSequenceClassification

id2label={0:"sadness",1:"joy",2:"love",3:"anger",4:"fear",5:"suprise"}
label2id={"sadness":0,"joy":1,"love":2,"anger":3,"fear":4,"suprise":5}

model=TFAutoModelForSequenceClassification.from_pretrained(
    model_ckpt, num_labels=6, id2label=id2label, label2id=label2id
)

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

## Preparing the datasets

In [ ]:
tf_train_set = model.prepare_tf_dataset(
    emotions_encoded["train"],
    shuffle = True,
    batch_size = 16,
    collate_fn = data_collator
)

tf_validation_set = model.prepare_tf_dataset(
    emotions_encoded["validation"],
    shuffle = True,
    batch_size = 16,
    collate_fn = data_collator
)

In [ ]:
import tensorflow as tf

optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer)

## Setting the callbacks

In [ ]:
from transformers.keras_callbacks import KerasMetricCallback
from transformers.keras_callbacks import PushToHubCallback

metric_callback=KerasMetricCallback(
    metric_fn=compute_metrics,
    eval_dataset = tf_validation_set
)

push_to_hub_callback=PushToHubCallback(
    output_dir = "emotion-analysis-with-distilbert",
    tokenizer = tokenizer
)

In [ ]:
model.fit(x=tf_train_set,
         validation_data=tf_validation_set,
         epochs=2,
         callbacks=[metric_callback, push_to_hub_callback])

# <b><div style='padding:15px;background-color:#850E35;color:white;border-radius:2px;font-size:110%;text-align: center'>5. Prediction </div></b>

In [ ]:
custom_text = "I watched a movie yesterday. It was really awesome"

In [ ]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis", 
                     model = "Tirendaz/emotion-analysis-with-distilbert")

In [ ]:
inputs=tokenizer(custom_text, return_tensors= "tf")
logits = model(**inputs).logits
predicted_class_id=int(tf.math.argmax(logits, axis=-1)[0])
model.config.id2label[predicted_class_id]

**Thanks for reading! If you enjoyed this notebook, don't forget to upvote** 👍

*Let's connect* [YouTube](http://youtube.com/tirendazacademy) | [Medium](http://tirendazacademy.medium.com) | [X](http://x.com/tirendazacademy) | [Linkedin](https://www.linkedin.com/in/tirendaz-academy) 😎

## Resource

- [NLP with Transformers](https://github.com/nlp-with-transformers/notebooks/blob/main/02_classification.ipynb)